In [2]:
import torchvision
import torch
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np

In [26]:
train_dataset_path = 'Dataset/train2017/'
test_dataset_path = 'Dataset/val2017'

In [28]:
mean = [0.4701, 0.4469, 0.4076]
std = [0.2330, 0.2280, 0.2315]

train_transforms = transforms.Compose ([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

In [30]:
train_data =  torchvision.datasets.ImageFolder(root = train_dataset_path, transform = train_transforms)
test_data = torchvision.datasets.ImageFolder(root = test_dataset_path, transform  = test_transforms)

In [31]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32
train_dataloader = DataLoader(dataset=train_data,
                             batch_size=BATCH_SIZE,
                             shuffle=True)
test_dataloader = DataLoader(dataset=val_data,
                       batch_size=BATCH_SIZE,
                       shuffle=False)

In [34]:
train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x1f75a334350>,
 <torch.utils.data.dataloader.DataLoader at 0x1f75befffd0>)

In [40]:
print(f"DataLoaders: {train_dataloader, val_dataloader}")
print(f"Length of the train_dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}...")
print(f"Length of the test_dataloader: {len(val_dataloader)} batches of {BATCH_SIZE}...")

DataLoaders: (<torch.utils.data.dataloader.DataLoader object at 0x000001F75A334350>, <torch.utils.data.dataloader.DataLoader object at 0x000001F75D1EF690>)
Length of the train_dataloader: 3697 batches of 32...
Length of the test_dataloader: 157 batches of 32...


In [44]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))

In [46]:
train_features_batch.shape, train_labels_batch.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

In [97]:
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_labels_batch[random_idx]
print(f"Image size: {img.shape}")
print(f"Label: {label}, label size: {label.shape}")

Image size: torch.Size([3, 224, 224])
Label: 0, label size: torch.Size([])


In [99]:
import torch.nn as nn

In [113]:
flatten_model = nn.Flatten()
x = train_features_batch[0]
output = flatten_model(x)
print(f"Shape before flattening: {x.shape}")
print(f"Shape after flattening: {output.shape}")

Shape before flattening: torch.Size([3, 224, 224])
Shape after flattening: torch.Size([3, 50176])


In [135]:
from pycocotools.coco import COCO
coco = COCO(os.path.join("Dataset/annotations_trainval2017/annotations/instances_train2017.json"))

loading annotations into memory...
Done (t=19.38s)
creating index...
index created!


NameError: name 'categories' is not defined

In [138]:
categories = coco.cats
n_classes = len(categories.keys())
class_names = [i[1]['name'] for i in categories.items()]

In [140]:
class CocoModelV0(nn.Module):
    def __init__(self,
                input_shape: int,
                hidden_units: int,
                output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape,
                     out_features=hidden_units),
            nn.Linear(in_features=hidden_units,
                     out_features=output_shape)
        )
    def forward(self, x):
        return self.layer_stack(x)

In [142]:
torch.manual_seed(42)
model_0 = CocoModelV0(
    input_shape = 224*224,
    hidden_units = 100,
    output_shape=len(class_names)
    
).to('cpu')
model_0

CocoModelV0(
  (layer_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=50176, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=80, bias=True)
  )
)

In [144]:
dummy_x = torch.rand([1, 3, 224, 224])
model_0(dummy_x)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x150528 and 50176x100)